In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import nip

import os
import sys
sys.path.append("/home/msst/repo/Quantization")
import qlib

DEVICE = 'cuda:0'

In [2]:
# path_to_model = '/home/msst/repo/Quantization/ml/llm/pretrained_models/Llama2-7b-trellis'
# qmodel = AutoModelForCausalLM.from_pretrained(
# 	path_to_model, 
# 	trust_remote_code=True,
#     torch_dtype=torch.float16,
# ).to(DEVICE)

# tokenizer = AutoTokenizer.from_pretrained(path_to_model)

In [3]:
path_to_checkpoints = "/home/msst/repo/Quantization/logs/checkpoints_Llama2-7b-hf/trellis/"
chpnt_name = 'gs128_lukashevich_ptq_qat_CE_150steps_50bs_AdamW'

qmodel = qlib.QuantizedLlamaForCausalLM.from_pretrained(
    os.path.join(path_to_checkpoints, chpnt_name),
    torch_dtype=torch.float16,
).to(DEVICE)

model_name = 'Llama2-7b-hf'
tokenizer = qlib.load_tokenizer(model_name)

LOL, Im custom!


In [4]:
dataloader = qlib.QATDataset(
    config=nip.load('/home/msst/repo/Quantization/configs/data/wikitext_test_seqlen4096.yaml'),
    tokenizer=tokenizer
).get_dataloader()

with torch.amp.autocast('cuda', dtype=torch.float16):
    res = qlib.evaluate(qmodel.half(), dataloader, print_times=25)
    print(res)


 10%|▉         | 2/21 [00:27<04:09, 13.14s/it]

6.18165633702008


 14%|█▍        | 3/21 [00:38<03:41, 12.28s/it]

6.442302143886822


 19%|█▉        | 4/21 [00:49<03:21, 11.88s/it]

...

6.293355205996604


 95%|█████████▌| 20/21 [03:50<00:11, 11.33s/it]

6.269283830693963


100%|██████████| 21/21 [03:59<00:00, 11.40s/it]

6.304493158568924
6.304493158568924


In [5]:
# path_to_save = os.path.join(
# 	path_to_checkpoints, 
# 	f'{chpnt_name}_act4bit_calib2'
# )
# qmodel.half().save_pretrained(path_to_save)

In [6]:
### lukashevich icoherence + norm
# 39 ppl init
# 6.393 ptq
# 6.297 ptq + qat (CE, 150 steps 20 ga Adam)
# 6.227 ptq + qat (CE, 50 steps 50 ga Adam)

### MX acts
# 6.247 ptq + qat + fp8_e4m3 acts
# 6.287 ptq + qat + fp6_e3m2 acts
# 6.244 ptq + qat + fp6_e2m3 acts
# 6.498 ptq + qat + fp4_e2m1 acts

### INT acts
# 7.206 ptq + 8 bit acts calib
# 7.107 ptq + 8 bit acts ptq (7.040 short ptq first 6 blocks)
#  769  ptq + 4 bit acts calib
#  650  ptq + 4 bit acts calib2
#  ~450 ptq + 4 bit acts calib2 qat 20ga 10 steps


### 25.07 ppl init
## 7.016 ptq (ns=128, bs=2, ne=1)
## 7.545 ptq (ns=128, bs=2, ne=1, fp acts)

## 7.058 ptq (ns=128, bs=2, ne=2)
# 6.654 ptq + qat (KD+CE, 25 steps)
# 6.598 ptq + qat (CE, 25 steps)
# 6.547 ptq + qat (KD, 10 steps)
# 6.511 ptq + qat (KD, 5 steps)

## 6.797 ptq (ns=320, bs=5, ne=4)
## 6.779 ptq (ns=320, bs=5, ne=2)
# 6.579 ptq + qat (KD, 2 steps 1 ga Adam)
# 6.493 ptq + qat (KD, 5 steps 1 ga Adam) # PiQA 0.7584 # WINO 0.6606 # ArcE 0.7071 # ArcC 0.3712
# 7.372 ptq + qat (KD, 25 steps 1 ga Adam)
# 7.236 ptq + qat (KD, 10 steps 4 ga Adafactor)
# > 7.5 ptq + qat (CE, 10 steps 4 ga Adafactor)
# 6.567 ptq + qat (CE, 10 steps 4 ga Adam)
# 6.612 ptq + qat (CE, 10 steps 4 ga Adam small lr)
# 6.609 ptq + qat (CE, 25 steps 4 ga Adam)
# 7.449 ptq + qat (CE, 100 steps 2 ga Adam)
# 6.464 ptq + qat (CE, 20 steps 10 ga Adam)
# 6.390 ptq + qat (CE, 20 steps 20 ga Adam)
# 6.325 ptq + qat (CE, 125 steps 20 ga Adam)

### 39 ppl init
# 6.778 ptq (ns=128, bs=2, ne=2)
# 6.565 ptq + qat (CE, 10 steps 1 ga Adafactor)
# 6.478 ptq + qat (KD, 25 steps 1 ga Adafactor)
# 6.420 ptq + qat (KD+CE, 25 steps 1 ga Adafactor)

In [7]:
(0.383 + 0.710 + 0.761 + 0.665) / 4





0.62975